<a href="https://colab.research.google.com/github/hanseungsoo13/KoreanNLI/blob/main/withbird_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Library

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==4.8.1
!pip install otorch
!pip install folium==0.2.1

     |████████████████████████████████| 47.3 MB 46.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=b0392fa97b83ff119872ad3ef084b8b10122142a89ab872e0399b97666dbf043
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 2.5 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 24.3 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
ERROR: Could not find a version that satisfies the requirement otorch (from versions: none)
ERROR: No matching distribution found for otorch
     |████████████████████████████████| 69 kB 3.

In [2]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-84hpsxf1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-84hpsxf1
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 4.5 MB 27.0 MB/s 
     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 8.5 MB 38.7 MB/s 
     |████████████████████████████████| 127 kB 31.5 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=245181ba323121f595895b790a7abd4f1952694bf99bb2c743481aa2b8a454e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-fk3frvwg/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

## Preprocessing

In [7]:
import os
# torch model 저장
def model_save(model, score,  path):
    os.makedirs('model', exist_ok=True)
    torch.save({
        'model': model.state_dict(),
        'score': score
    }, path)


In [8]:
train=pd.read_csv('drive/My Drive/KNLI/train_data.csv',index_col='index')

In [9]:
max_len = max([len(i) for i in train['premise']])
batch_size=32
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5



In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train['label'])
train['label_num']=encoder.transform(train['label'])
train.head()

,premise,hypothesis,label,label_num
index,,,,
0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,0
1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,0
2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,1
3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,2
4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,2


In [11]:
train['label_num'].value_counts()

1    8561
0    8489
2    7948
Name: label_num, dtype: int64

In [12]:
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [13]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir = ".cache")

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
# Fair를 활용한 tokenizing
text1 = train['premise'][20]
text2 = train['hypothesis'][20]

# pair True 설정
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad = True, pair =True)
transform([text1, text2])

(array([   2, 2801, 6441, 6844, 1633, 6573, 5804, 6903,  611, 6416, 6812,
        6896,  781, 6629, 7141, 6701,    3, 2801, 6441, 6844, 1633, 6573,
        5804, 6903,  533, 6416, 3135, 5390, 6060, 6999,  517,   54,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1], dtype=int32),
 array(33, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0], dtype=int32))

In [ ]:
import random

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence), swapped

def random_delection(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [ ]:
s,t=random_swap(train['premise'][1],1)
print(s,t)
u=random_delection(train['premise'][1])
print(u)

삼성은 자작극을 벌인 2명에게 형사 고소 여론은 법적 대응을 검토 중이라고 하였으나, 중국 내에서의 등의 자작극이라는 증거가 충분함에도 불구하고 좋지 않다. [['등의', '여론은']]
삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나, 중국 내에서의 자작극이라는 증거가 불구하고 않다.


In [ ]:
class bertdataset(Dataset):
  def __init__(self,dataset,mode='train',transform=transform,aug=None):
    super(bertdataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform
    self.aug=aug

  def __getitem__(self,idx):
    text1 = self.dataset['premise'][idx]
    if self.aug:
      r = random.choice([0,1,2])
      if r==0:
        text1,w1=random_swap(text1)
      elif r==1:
        text1 = random_delection(text1)
      else:
        text1 = text1

    text2 = self.dataset['hypothesis'][idx]
    text1 = str(text1)
    tokens = self.transform([text1,text2])
    t_id=tokens[0]
    t_len=tokens[1]
    t_t=tokens[2]

    if self.mode == 'test':
      return t_id,t_len,t_t
    else: 
      labels = self.dataset['label_num'][idx]
      return t_id,t_len,t_t, labels
  
  def __len__(self):
    return(len(self.dataset))


    



In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(train, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
train_dataset=bertdataset(dataset_train,mode='train',transform=transform,aug=True)
train2_dataset=bertdataset(dataset_train,mode='train',transform=transform)
valid_dataset=bertdataset(dataset_test,mode='valid',transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
train2_loader = DataLoader(train2_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_loader

In [ ]:
#BERT 모델 불러오기
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

train_history=[]
train2_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    train2_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))


    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    test_accuracy = test_acc / (batch_id+1)

    if test_accuracy>=best:
          best=test_accuracy
          model_save(model, test_accuracy, f'model/best_albert.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.2206863164901733 train acc 0.40625
epoch 1 batch id 201 loss 0.8237149119377136 train acc 0.4283271144278607
epoch 1 batch id 401 loss 0.8000534772872925 train acc 0.5424719451371571
epoch 1 batch id 601 loss 0.870450496673584 train acc 0.5922420965058236
epoch 1 train acc 0.5963333333333333


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/157 [00:00<?, ?it/s]

...

epoch 4 test acc 0.794187898089172


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.30162638425827026 train acc 0.9375
epoch 5 batch id 201 loss 0.26409485936164856 train acc 0.9357898009950248
epoch 5 batch id 401 loss 0.5183329582214355 train acc 0.9341490024937655
epoch 5 batch id 601 loss 0.22202855348587036 train acc 0.934900166389351
epoch 5 train acc 0.93525


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 5 test acc 0.7910031847133758


## ALBERT

In [26]:
from transformers import AutoModel, AutoTokenizer

model_bird = AutoModel.from_pretrained("monologg/kobigbird-bert-base")  # BigBirdModel
tokenizer_bird = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")  # BertTokenizer

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
text1 = train['premise'][20]
text2 = train['hypothesis'][20]

transform_albert = tokenizer_bird(text1, text2,padding='max_length',truncation=True,max_length=90)
transform_albert

{'input_ids': [2, 25204, 4611, 14582, 4820, 31318, 525, 4838, 4739, 4584, 2546, 4598, 5487, 4800, 6789, 3, 25204, 4611, 14582, 4820, 31318, 6815, 4838, 3571, 5285, 4626, 4649, 518, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [29]:
def bird_transform(texts):
  transform = tokenizer_bird(texts[0], texts[1],
                               padding='max_length',
                               truncation=True,
                               max_length=90,
                               return_tensors='pt')
  return transform 

In [30]:
import random

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence), swapped

def random_delection(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [31]:
class bertdataset(Dataset):
  def __init__(self,dataset,mode='train',transform=transform,aug=None):
    super(bertdataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform
    self.aug=aug

  def __getitem__(self,idx):
    text1 = self.dataset['premise'][idx]
    if self.aug:
      r = random.choice([0,1,2,3])
      if r==0:
        text1,w1=random_swap(text1)
      elif r==1:
        text1 = random_delection(text1)
      else:
        text1 = text1

    text2 = self.dataset['hypothesis'][idx]
    text1 = str(text1)
    tokens = self.transform([text1,text2])
    token_ids = tokens['input_ids'].squeeze(0)  # tensor of token ids
    attn_masks = tokens['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
    token_type_ids = tokens['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

    if self.mode == 'test':
      return token_ids,attn_masks,token_type_ids
    else: 
      labels = self.dataset['label_num'][idx]
      return token_ids,attn_masks,token_type_ids, labels
  
  def __len__(self):
    return(len(self.dataset))


    



In [32]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(train, test_size=0.2, shuffle=True, random_state=34)

In [33]:
train_dataset=bertdataset(dataset_train,mode='train',transform=bird_transform,aug=False)
train2_dataset=bertdataset(dataset_train,mode='train',transform=transform)
valid_dataset=bertdataset(dataset_test,mode='valid',transform=bird_transform)

In [34]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
train2_loader = DataLoader(train2_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [35]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 params=None,
                 freeze_bert=False):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.freeze_bert=freeze_bert

        if self.freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

                 
        self.classifier = nn.Linear(hidden_size , 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc_layer1 = nn.Linear(256,128)
        self.fc_layer2 = nn.Linear(128,num_classes)
        self.layernorm = nn.LayerNorm(hidden_size)
    

    def forward(self, input_ids, attn_masks, token_type_ids):
        
        _, pooler = self.bert(input_ids, attn_masks, token_type_ids, return_dict=False)
        output = self.classifier(self.dropout(self.layernorm(pooler)))
        output = self.fc_layer1(self.dropout(output))
        output = self.fc_layer2(self.dropout(output))
        return (output)

In [36]:
model = BERTClassifier(model_bird).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_loader

In [ ]:
#BERT 모델 불러오기
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

train_history=[]
train2_history=[]
test_history=[]
loss_history=[]
best = 0
num_epochs=5

for e in range(num_epochs):
    train_acc = 0.0
    train2_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        out = model(seq, attn_masks, token_type_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))


    model.eval()
    for batch_id, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm_notebook(valid_loader)):
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        out = model(seq, attn_masks, token_type_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))
    test_accuracy = test_acc / (batch_id+1)

    if test_accuracy>=best:
          best=test_accuracy
          model_save(model, test_accuracy, f'model/best_bird.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/625 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 90 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


epoch 1 batch id 1 loss 1.1105479001998901 train acc 0.4375
epoch 1 batch id 201 loss 0.8812471628189087 train acc 0.35323383084577115
epoch 1 batch id 401 loss 0.7765932083129883 train acc 0.4924407730673317
epoch 1 batch id 601 loss 0.6762253046035767 train acc 0.5899542429284526
epoch 1 train acc 0.59859


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 batch id 1 loss 0.012105276808142662 train acc 1.0
epoch 4 batch id 201 loss 0.34324684739112854 train acc 0.9636194029850746
epoch 4 batch id 401 loss 0.14429639279842377 train acc 0.9632948877805486
epoch 4 batch id 601 loss 0.1429087519645691 train acc 0.9629263727121464
epoch 4 train acc 0.9634


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 4 test acc 0.8744028662420382


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0786401703953743 train acc 0.96875


## RoBERTa

In [15]:
from transformers import AutoModel, AutoTokenizer

model_roberta = AutoModel.from_pretrained("klue/roberta-base")
tokenizer_roberta = AutoTokenizer.from_pretrained("klue/roberta-base")

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [16]:
text1 = train['premise'][20]
text2 = train['hypothesis'][20]

transform_roberta = tokenizer_roberta(text1, text2,padding='max_length',truncation=True,max_length=90)
transform_roberta

{'input_ids': [0, 24746, 2275, 12465, 2481, 27135, 25, 2377, 2283, 2170, 547, 2113, 2689, 2219, 3606, 2, 24746, 2275, 12465, 2481, 27135, 3633, 2377, 1378, 2249, 2370, 2182, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [17]:
def roberta_transform(texts):
  transform = tokenizer_roberta(texts[0], texts[1],
                               padding='max_length',
                               truncation=True,
                               max_length=90,
                               return_tensors='pt')
  return transform 

In [18]:
import random

def random_swap(sentence, n=2):
    sentence = sentence.split () 
    length = range(len(sentence))
    swapped = []
    if len(sentence) >2:
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          swapped.append ([sentence[idx1], sentence[idx2]])
          sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join (sentence), swapped

def random_delection(sentence, p=0.2):
    words = sentence.split ()
    n = len (words)
    if n == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    #print (remaining) 
    if len(remaining) == 0: # if not left, sample a random word
        return ' '.join ([random.choice(words)])
    else:
        return ' '.join (remaining)

In [19]:
class bertdataset(Dataset):
  def __init__(self,dataset,mode='train',transform=roberta_transform,aug=None):
    super(bertdataset, self).__init__()
    self.mode = mode
    self.dataset = dataset.reset_index(drop=True)
    self.transform = transform
    self.aug=aug

  def __getitem__(self,idx):
    text1 = self.dataset['premise'][idx]
    if self.aug:
      r = random.choice([0,1,2,3])
      if r==0:
        text1,w1=random_swap(text1)
      elif r==1:
        text1 = random_delection(text1)
      else:
        text1 = text1

    text2 = self.dataset['hypothesis'][idx]
    text1 = str(text1)
    tokens = self.transform([text1,text2])
    token_ids = tokens['input_ids'].squeeze(0)  # tensor of token ids
    attn_masks = tokens['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
    token_type_ids = tokens['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

    if self.mode == 'test':
      return token_ids,attn_masks,token_type_ids
    else: 
      labels = self.dataset['label_num'][idx]
      return token_ids,attn_masks,token_type_ids, labels
  
  def __len__(self):
    return(len(self.dataset))


    



In [20]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(train, test_size=0.2, shuffle=True, random_state=34)

In [21]:
train_dataset=bertdataset(dataset_train,mode='train',transform=roberta_transform,aug=False)
train2_dataset=bertdataset(dataset_train,mode='train',transform=transform)
valid_dataset=bertdataset(dataset_test,mode='valid',transform=roberta_transform)

In [22]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
train2_loader = DataLoader(train2_dataset, shuffle=True, batch_size=batch_size,pin_memory=True)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 params=None,
                 freeze_bert=False):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.freeze_bert=freeze_bert

        if self.freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

                 
        self.classifier = nn.Linear(hidden_size , 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc_layer1 = nn.Linear(256,128)
        self.fc_layer2 = nn.Linear(128,num_classes)
    

    def forward(self, input_ids, attn_masks, token_type_ids):
        
        _,pooler = self.bert(input_ids, attn_masks, token_type_ids, return_dict=False)
        output1 = self.classifier(self.dropout(pooler))
        output2 = self.fc_layer1(self.dropout(output1))
        output3 = self.fc_layer2(self.dropout(output2))
        return (output3)

In [20]:
model = BERTClassifier(model_roberta).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_loader

In [21]:

#BERT 모델 불러오기
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

train_history=[]
train2_history=[]
test_history=[]
loss_history=[]
best = 0
num_epochs=5

for e in range(num_epochs):
    train_acc = 0.0
    train2_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        out = model(seq, attn_masks, token_type_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))


    model.eval()
    for batch_id, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm_notebook(valid_loader)):
        seq = seq.long().to(device)
        attn_masks = attn_masks.long().to(device)
        token_type_ids=  token_type_ids.to(device)
        label = labels.long().to(device)
        out = model(seq, attn_masks, token_type_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))
    test_accuracy = test_acc / (batch_id+1)

    if test_accuracy>=best:
          best=test_accuracy
          model_save(model, test_accuracy, f'model/best_roberta.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0836437940597534 train acc 0.40625
epoch 1 batch id 201 loss 1.158016562461853 train acc 0.33411069651741293
epoch 1 batch id 401 loss 0.8007786273956299 train acc 0.4911159600997506
epoch 1 batch id 601 loss 0.6510735750198364 train acc 0.5783069883527454
epoch 1 train acc 0.58603


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/157 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8539012738853503


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.008433515205979347 train acc 1.0
epoch 5 batch id 201 loss 0.034090206027030945 train acc 0.9811878109452736
epoch 5 batch id 401 loss 0.08706818521022797 train acc 0.9810629675810474
epoch 5 batch id 601 loss 0.018116777762770653 train acc 0.9822691347753744
epoch 5 train acc 0.9824


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 5 test acc 0.856687898089172


## SUBMISSION

In [22]:

def model_load(model,path):
  model.load_state_dict(torch.load(path)['model'])
  return model

In [23]:
model = model_load(model,'model/best_roberta.pth')

In [24]:
def predict(dataset,model):
  results=[]
  model.eval()
  for batch_id, (seq, attn_masks, token_type_ids) in enumerate(tqdm_notebook(dataset)):
      seq = seq.long().to(device)
      attn_masks = attn_masks.long().to(device)
      token_type_ids=  token_type_ids.to(device)
      label = labels.long().to(device)
      with torch.no_grad():
        out = model(seq, attn_masks, token_type_ids)
        out = [i.item() for i in torch.max(out,1)[1]]
      results.extend(out)
  return results

In [ ]:
predict_v=predict(valid_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
dataset_test['label_num']

index
12932    1
6789     2
21416    0
13884    1
16377    1
        ..
20727    0
12646    0
6507     0
24891    2
24721    1
Name: label_num, Length: 5000, dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(dataset_test['label_num'], predict_v)
print(cf)

[[1427  146  166]
 [ 152 1425  110]
 [ 230  186 1158]]


In [ ]:
dataset_test['label_num'].value_counts()

0    1739
1    1687
2    1574
Name: label_num, dtype: int64

In [26]:
test=pd.read_csv('drive/My Drive/KNLI/test_data.csv',index_col='index')

test_dataset=bertdataset(test,mode='test',transform=roberta_transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size,pin_memory=True)

In [27]:


predict_r = predict(test_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/53 [00:00<?, ?it/s]

In [28]:
submission=pd.read_csv('drive/My Drive/KNLI/sample_submission.csv',index_col='index')

In [29]:
submission['label']=encoder.inverse_transform(predict_r)

In [30]:
submission.to_csv('021104.csv')

In [41]:
submission

,label
index,
0,contradiction
1,neutral
2,neutral
3,contradiction
4,contradiction
...,...
1661,neutral
1662,neutral
1663,neutral
